#Exame Prático: Spectrum Shades LLC

A Spectrum Shades LLC é uma fornecedora de soluções de coloração para concreto, oferecendo uma ampla variedade de pigmentos e sistemas de coloração utilizados em diversas aplicações, como concreto decorativo, pré-moldado e blocos de concreto. A empresa se orgulha de fornecer colorantes de alta qualidade que atendem às necessidades específicas de seus variados clientes, incluindo empreiteiros, arquitetos e construtoras.


Recentemente, a empresa observou um aumento no número de reclamações de clientes relacionadas à inconsistência na qualidade das cores de seus produtos. Essas discrepâncias têm levado à queda na satisfação dos clientes e ao possível aumento nas devoluções de produtos.

Ao identificar e mitigar os fatores que causam variações de cor, a empresa pode melhorar a confiabilidade dos produtos, reduzir reclamações e minimizar as taxas de devolução.


Você faz parte da equipe de análise de dados encarregada de fornecer insights acionáveis para ajudar a Spectrum Shades LLC a resolver os problemas de inconsistência na coloração e aumentar a satisfação dos clientes.

#Tarefa 1: Validação e Limpeza de Dados

Antes de iniciar qualquer análise, é necessário garantir que os dados estão corretos e condizem com o esperado.

Sabe-se que há alguns problemas na tabela production_data, e a equipe de dados forneceu a seguinte descrição dos dados.

🎯 Escreva uma consulta para garantir que os dados correspondem à descrição fornecida, incluindo a identificação e correção de todos os valores inválidos. É obrigatório seguir os nomes das colunas e os critérios descritos abaixo.

*   O ponto de partida é o arquivo "production_data.csv".
*   O resultado final deve ser um DataFrame chamado clean_data.
*   Todos os nomes de colunas e valores devem obedecer às regras da tabela a seguir:

| Nome da Coluna          | Critério                                                                                                                                                                    |
| ----------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `batch_id`              | Discreto. Identificador de cada lote. **Não pode haver valores ausentes.**                                                                                                  |
| `production_date`       | Data. Data em que o lote foi produzido.                                                                                                                                     |
| `raw_material_supplier` | Categórico. Fornecedor da matéria-prima. (1 = `'national_supplier'`, 2 = `'international_supplier'`).<br>Valores ausentes devem ser substituídos por `'national_supplier'`. |
| `pigment_type`          | Nominal. Tipo de pigmento utilizado. Valores válidos: `['type_a', 'type_b', 'type_c']`. <br>Valores ausentes devem ser substituídos por `'other'`.                          |
| `pigment_quantity`      | Contínuo. Quantidade de pigmento adicionada (em kg). Intervalo válido: **1 a 100**.<br>Valores ausentes devem ser substituídos pela **mediana**.                            |
| `mixing_time`           | Contínuo. Duração do processo de mistura (em minutos). <br>Valores ausentes devem ser substituídos pela **média**, **arredondada para 2 casas decimais**.                   |
| `mixing_speed`          | Categórico. Velocidade do processo de mistura: `'Low'`, `'Medium'`, `'High'`. <br>Valores ausentes devem ser substituídos por `'Not Specified'`.                            |
| `product_quality_score` | Contínuo. Nota geral da qualidade do produto final (escala de 1 a 10).<br>Valores ausentes devem ser substituídos pela **média**, **arredondada para 2 casas decimais**.    |


In [6]:
import pandas as pd
from IPython.display import display

# Carregando o dataset
clean_data = pd.read_csv('production_data.csv')

# Preenchendo valores ausentes com regras definidas
clean_data['raw_material_supplier'] = clean_data['raw_material_supplier'].fillna('national_supplier')
clean_data['pigment_type'] = clean_data['pigment_type'].fillna('other')

median_pigment_quantity = clean_data['pigment_quantity'].median()
clean_data['pigment_quantity'] = clean_data['pigment_quantity'].fillna(median_pigment_quantity)

mean_mixing_time = clean_data['mixing_time'].mean()
clean_data['mixing_time'] = clean_data['mixing_time'].fillna(round(mean_mixing_time, 2))

clean_data['mixing_speed'] = clean_data['mixing_speed'].fillna('Not Specified')

mean_quality_score = clean_data['product_quality_score'].mean()
clean_data['product_quality_score'] = clean_data['product_quality_score'].fillna(round(mean_quality_score, 2))

# Conversão de tipos
clean_data['production_date'] = pd.to_datetime(clean_data['production_date'])

# Normalização de texto
clean_data['pigment_type'] = clean_data['pigment_type'].str.lower()
clean_data['mixing_speed'] = clean_data['mixing_speed'].replace('-', 'Not Specified')

# Mostrar resultado final formatado
pd.set_option('display.max_columns', None)

display(clean_data.head(10).style
        .format({
            'pigment_quantity': '{:.2f}',
            'mixing_time': '{:.2f}',
            'product_quality_score': '{:.2f}'
        })
        .set_caption("✅ Primeiras 10 Linhas do DataFrame `clean_data` (após limpeza)"))


,batch_id,production_date,raw_material_supplier,pigment_type,pigment_quantity,mixing_time,mixing_speed,product_quality_score
0,1,2024-06-25 00:00:00,1,type_a,42.82,22.56,High,7.17
1,2,2023-11-23 00:00:00,2,type_b,42.87,44.97,High,6.85
2,3,2024-02-18 00:00:00,2,type_b,33.25,52.62,High,5.66
3,4,2023-11-11 00:00:00,1,type_a,52.53,24.96,High,6.99
4,5,2024-04-11 00:00:00,2,type_c,36.21,25.37,High,7.10
5,6,2024-02-14 00:00:00,2,type_b,29.99,45.96,Medium,6.14
6,7,2023-08-25 00:00:00,2,type_a,35.94,32.13,High,5.74
7,8,2024-04-26 00:00:00,2,type_a,40.50,7.75,Medium,5.51
8,9,2024-07-06 00:00:00,2,type_a,36.02,35.28,Low,4.96
9,10,2023-11-21 00:00:00,1,type_a,42.89,18.61,High,9.88


# Tarefa 2: Agrupamento por Fornecedor

Você quer entender como o tipo de fornecedor e a quantidade de materiais influenciam os atributos do produto final.

🎯 Calcule a média do product_quality_score e da pigment_quantity, agrupando pelos valores de raw_material_supplier.

*   Você deve utilizar os dados do arquivo production_data.csv.
*   O resultado deve ser um DataFrame chamado aggregated_data.
*   O DataFrame final deve conter três colunas: raw_material_supplier, avg_product_quality_score, avg_pigment_quantity
*   Os valores devem ser arredondados para 2 casas decimais.

    

In [8]:
# Agrupar por 'raw_material_supplier' e calcular médias
aggregated_data = clean_data.groupby('raw_material_supplier')[['product_quality_score', 'pigment_quantity']].mean().reset_index()

# Renomear colunas conforme especificado
aggregated_data.rename(columns={
    'product_quality_score': 'avg_product_quality_score',
    'pigment_quantity': 'avg_pigment_quantity'
}, inplace=True)

# Arredondar para 2 casas decimais
aggregated_data['avg_product_quality_score'] = aggregated_data['avg_product_quality_score'].round(2)
aggregated_data['avg_pigment_quantity'] = aggregated_data['avg_pigment_quantity'].round(2)

# Exibir resultado de forma amigável
from IPython.display import display

display(aggregated_data.style
        .format({'avg_product_quality_score': '{:.2f}', 'avg_pigment_quantity': '{:.2f}'})
        .set_caption("📊 Médias por Tipo de Fornecedor"))


,raw_material_supplier,avg_product_quality_score,avg_pigment_quantity
0,1,8.02,44.73
1,2,5.97,34.91


#Tarefa 3: Filtro por Fornecedor e Quantidade de Pigmento

Para entender melhor os fatores que influenciam a qualidade do produto, você quer filtrar os dados para visualizar a média da pontuação de qualidade em um conjunto específico de registros.

🎯 Identifique a média do product_quality_score para os lotes que possuem:

*   raw_material_supplier igual a 2 (ou seja, fornecedor internacional)
*   pigment_quantity maior que 35 kg

🎯 Utilize os dados originais do arquivo production_data.csv, e não o resultado da Tarefa 2.

*   O resultado deve ser um DataFrame chamado pigment_data
*   Ele deve conter apenas 1 linha e 3 colunas: raw_material_supplier, pigment_quantity, avg_product_quality_score
*   Os valores devem ser arredondados para 2 casas decimais, quando apropriado.





    



    

In [10]:
# Filtrar dados com fornecedor internacional e pigmento > 35 kg
filtered_data = clean_data[(clean_data['raw_material_supplier'] == 2) & (clean_data['pigment_quantity'] > 35)]

# Calcular médias
avg_quality = filtered_data['product_quality_score'].mean()
avg_quantity = filtered_data['pigment_quantity'].mean()

# Criar DataFrame formatado
pigment_data = pd.DataFrame({
    'raw_material_supplier': [2],
    'pigment_quantity': [round(avg_quantity, 2)],
    'avg_product_quality_score': [round(avg_quality, 2)]
})

# Exibir resultado de forma bonita
from IPython.display import display

display(pigment_data.style
        .format({
            'pigment_quantity': '{:.2f}',
            'avg_product_quality_score': '{:.2f}'
        })
        .set_caption("🔍 Lotes com Fornecedor Internacional e Pigmento > 35 kg"))


,raw_material_supplier,pigment_quantity,avg_product_quality_score
0,2,39.01,5.97


#Tarefa 4: Estatísticas Descritivas e Correlação

Para avançar com análises mais detalhadas, você precisa analisar como diversos fatores se relacionam com a qualidade do produto.

🎯 Comece calculando a média e o desvio padrão para as seguintes colunas:

*   pigment_quantity
*   product_quality_score

Essas estatísticas ajudam a entender a tendência central e a variabilidade dos dados relacionados à qualidade do produto.

🎯 Em seguida, calcule o coeficiente de correlação de Pearson entre as variáveis:

*   pigment_quantity
*   product_quality_score

🎯 Esse coeficiente indicará a força e direção da relação entre a quantidade de pigmento e a qualidade do produto final.

*   Use os dados do arquivo production_data.csv.
  *   Calcule os seguintes valores:
  *   product_quality_score_mean
  *   product_quality_score_sd
  *   pigment_quantity_mean
  *   pigment_quantity_sd
  *   corr_coef
*   O resultado deve ser um DataFrame chamado product_quality.
*   Ele deve conter as colunas mencionadas acima.
*   Todos os valores devem ser arredondados para 2 casas decimais.

In [13]:
# Cálculo das estatísticas descritivas
product_quality_score_mean = clean_data['product_quality_score'].mean()
product_quality_score_sd = clean_data['product_quality_score'].std()
pigment_quantity_mean = clean_data['pigment_quantity'].mean()
pigment_quantity_sd = clean_data['pigment_quantity'].std()

# Cálculo da correlação de Pearson
corr_coef = clean_data['pigment_quantity'].corr(
    clean_data['product_quality_score'], method='pearson'
)

# Montar DataFrame com os resultados
product_quality = pd.DataFrame({
    'product_quality_score_mean': [round(product_quality_score_mean, 2)],
    'product_quality_score_sd': [round(product_quality_score_sd, 2)],
    'pigment_quantity_mean': [round(pigment_quantity_mean, 2)],
    'pigment_quantity_sd': [round(pigment_quantity_sd, 2)],
    'corr_coef': [round(corr_coef, 2)]
})

# Exibir resultado de forma organizada
from IPython.display import display

display(product_quality.style
        .format({
            'product_quality_score_mean': '{:.2f}',
            'product_quality_score_sd': '{:.2f}',
            'pigment_quantity_mean': '{:.2f}',
            'pigment_quantity_sd': '{:.2f}',
            'corr_coef': '{:.2f}'
        })
        .set_caption("📈 Estatísticas e Correlação Entre Quantidade de Pigmento e Qualidade do Produto"))



,product_quality_score_mean,product_quality_score_sd,pigment_quantity_mean,pigment_quantity_sd,corr_coef
0,6.68,1.39,38.35,6.83,0.49
